In [2]:
"""Calibrate LWR with Gipps"""
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import os
from flow.core.macroscopic import ARZ
from flow.core.macroscopic.lwr import PARAMS as ARZ_PARAMS
from flow.core.macroscopic.utils import run
from flow.core.macroscopic.Calibration.aggregate_micro_to_macro import agg_func
params = ARZ_PARAMS.copy()

dir = "./Gipps/"

L = 2000
dx = 10
params['length'] = L
params['dx'] = dx
params['v_max_max'] = 100
params['rho_max_max'] = 300
params['total_time'] = 300
params["boundary_conditions"] = "loop"

Avg_pos = pd.DataFrame()
Avg_speed = pd.DataFrame()
file_list = os.listdir(dir)
for file in file_list:
    if "pos" in file:
        T = pd.read_csv(dir + file)
        Avg_pos = pd.concat([Avg_pos, T])
    if "speed" in file:
        S = pd.read_csv(dir + file)
        Avg_speed = pd.concat([Avg_speed, S])

## mean positions from csv
Pos = Avg_pos.groupby("time").mean()
## mean velocities from csv
Speeds = Avg_speed.groupby("time").mean()
# display(Pos.head())
# display(Speeds.head())


In [ ]:
# single 
## get densities amd velocities -> at specific time step t and t+1
t1 = 20
row = np.array(Pos.loc[t1])
speed = np.array(Speeds.loc[t1])
rho_init, vel_init = agg_func(row, speed, L, dx)
params['initial_conditions'] = rho_init

t2 = 21
row = np.array(Pos.iloc[t2])
speed = np.array(Speeds.iloc[t2])
rho_final, vel_final = agg_func(row, speed, L, dx)
# ##################################################################
display(Pos.iloc[[t1]])
# display(Pos.iloc[[t2]])

In [8]:
params['rho_max'] =1
params['v_max'] = 11
params['CFL'] = 0.95
params['initial_conditions'] = (rho_init,vel_init)
params['dt'] = 0.1
params['tau'] = 0.05
env = ARZ(params)
obs = env.reset()
obs, _, _, _ = env.step(rl_actions = params['v_max']) # one time step
# obs[:int(len(obs)/2)]
obs

/Users/gilbertbahatij/flow/flow/core/macroscopic/arz.py:549: RuntimeWarning: invalid value encountered in true_divide
  return (y_value / density) + self.ve(density)
/Users/gilbertbahatij/flow/flow/core/macroscopic/arz.py:427: RuntimeWarning: invalid value encountered in minimum
  q = np.minimum(d, s)
/Users/gilbertbahatij/anaconda3/envs/flow/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


array([0.        ,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan, 0.00083333, 0.00089517, 0.00083333, 0.00097325,
       0.00083334, 0.00083333, 0.00166666, 0.00166667, 0.00166667,
       0.00166667, 0.00166667, 0.00166667, 0.00083334, 0.00083333,
       0.00083333, 0.00083333, 0.00083333, 0.00083333, 0.00083333,
       0.00083333, 0.00083333, 0.00083333, 0.00083333, 0.00083333,
       0.00166666, 0.00179074, 0.00166667, 0.00154259, 0.00083334,
       0.00083333, 0.00166666, 0.00166667, 0.00083334, 0.00083333,
       0.00083333, 0.00083333, 0.00083333, 0.00083333, 0.00088591,
       0.00083333, 0.00079783, 0.00083333, 0.00197715, 0.00166667,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan, 0.00083333, 0.00083333, 0.00166666,
              nan,        nan,        nan,        nan, 0.00083